# **Libraries**

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

from feature_engine.imputation import ArbitraryNumberImputer

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_auc_score


import warnings
import time

# **Display**

In [5]:
%matplotlib inline

pd.options.display.max_rows = 300000
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 500

warnings.filterwarnings("ignore")

warnings.simplefilter(action="ignore", category=FutureWarning)

pd.set_option('display.max_rows', 200)

# **Load Data**

In [37]:
pd.set_option('use_inf_as_na', True)

data = pd.read_csv(
    r"C:\Users\Dell\Documents\AI\Risk\Data\Data\data 27.csv",
    index_col=False
)

## **Random Sampling**

In [38]:
data = data.sample(frac=0.2, random_state=42)

## **Remove Infinity Values**

In [39]:
data = data.replace([np.inf, -np.inf], np.nan)

## **Variables**

In [40]:
random_state = 101
target = 'TARGET'

## **Imputation**

In [41]:
ani = ArbitraryNumberImputer(arbitrary_number=-99999)
ani.fit(data)
data = ani.transform(data)

## **Train Test Split**

In [42]:
X = data.drop(target, axis=1)
y = data[target]

X, y = shuffle(X, y, random_state=random_state)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=random_state)

## **Modeling**

In [43]:
lg_model = LogisticRegression(random_state=random_state, max_iter=5000)
lg_pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('lg', lg_model)
])

rf_model = RandomForestClassifier(random_state=random_state)
rf_pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('random_forest', rf_model)
])

lgbm_model = LGBMClassifier(random_state=random_state, verbose=0)
lgbm_pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('lgbm', lgbm_model)
])

pipelines = {
    "Logistic Regression": lg_pipeline,
    "Random Forest": rf_pipeline,
    "LightGBM": lgbm_pipeline
}

for name, pipeline in pipelines.items():
    start_time = time.time()

    # Get cross-validated predicted probabilities for positive class
    y_pred_proba = cross_val_predict(pipeline, X, y, cv=10, method='predict_proba')[:, 1]
    
    # Calculate ROC AUC
    roc_auc = roc_auc_score(y, y_pred_proba)
    
    end_time = time.time()
    elapsed_time = (end_time - start_time) / 60  # Convert time to minutes

    print(f"{name}: ROC AUC = {roc_auc:.2f} ({elapsed_time:.2f} minutes)")


Logistic Regression: ROC AUC = 0.69 (0.09 minutes)
Random Forest: ROC AUC = 0.72 (17.49 minutes)
LightGBM: ROC AUC = 0.75 (3.79 minutes)
